# Lecture 4 ALU Design

### Important Terms:
* __Instruction Fetch__:  IF; fetch from memory 
* __Instruction Decode__: ID; figure out the instruction, from register
* __Execution__: EX; ALU performs the job  
* __Memory__: MEM ; data memory (storage)
* __Write Back__: WB ; write back to register files

Today look at the design of ALU, and the execution mechanism

### Slide Note

Arithmetic on Integers (Addition, subtraction, multiplication, division, overflow); floating-point real numbers (representation). <br>
#### Integer Addition
Addition:  Operand1 + operand2 + CarryIn -> Result + CarryOut <br>
Overflow if out of range: 
* Adding +ve and -ve operands, no overflow 
* Adding +ve and +ve operands but get -ve (sign bit 1), overflow = 1
* Adding -ve and -ve operands but get +ve (sign bit 0), overflow = 1 
#### Integer Subtraction
Subtraction is converted to addition: a-b is equivalent to a+(-b). <br>
Overflow if out of range:
* Subtracting two +ve or two -ve operands, no overflow 
* Subtracting +ve from -ve operand but get +ve (sign bit 0), overflow = 1 
* Subtracting -ve from +ve operand but gets -ve (sign bit 1), overflow = 1
#### Dealing with Overflow 
Some language (C) ignores overflow; Some language requires exception (i.e. save PC in exception program counter register, jump to predefined handler address, and finally return after correction action).

![alu](alu.png)

### One-Bit ALU 
1-bit operands performing AND, OR, and ADD <br>
![1-bit](1-bit-alu.png)
The 1-bit ALU has OR gate, AND gate, NOT gate, a multiplexer and a full adder. <br>
First we look at the 1-bit full adder. 
![1-bit-add](1-bit-adder.png)


Formula: (Ci as CarryIn, Co as carryout) 
```
Co = (!a & b & Ci)|(a & !b & Ci)|(a & b & !Ci)|( a & b & Ci)
Co = (b & Ci) | (a & Ci) | (a & b) 
```
This is thus a 3 AND gate going to 3 OR gate

### Ripple Carry ALU, 32-bit ALU
![32](32-bit.png)
The 32-bit ALU is constructed via 32 1-bit ALUs, each handling one digit and takes the carryout as the carry in value of the next ALU. 

Add Subtraction into the ALU: we need to add an invertor to the input and add 1.
```
a - b = a + (~b + 1) 
```
![alu-sub](alu-sub.png) 
When subtraction is required, we need to also set carry in value of the the first ALU (ALU0) to 1, since this is 2's complement. 

#### Overflow Detection 
![over](overflow.png)
For N-bit ALU, 
```
Overflow = Ci[N-1] XOR Co[N-1] 
```
check the ALU 31. (carry in of ALU 31 and carry out of ALU 31) and find the XOR, if XOR gives 1, overflow is detected. 

#### Zero Detection 
* Conditional Branches 
* NOR gate for all the bits). Any non-zero result will cause zero detection output to be zero. 

#### NOR 
        (~a & ~b)
![NOR](nor.png)
        

### SLT implementation
* SLT produces a 1 if rs < rt, and 0 otherwise.
    * all but least significant bit will be 0
    * how do we set the least significant bit?
    * can we use subtraction? 
    * set the least significant bit to sign-bit of (rs-rt)
* New input: LESS
* New output: SET 
![slt](slt.png)
![msb](msb.png)
For all but the most significant ALU, we have the __less__ option that directly goes to the result. This means that if option 3 is picked from the MUX, then ALU will directly use the value of __less__. <br>
For the most significant ALU, besides less, we have the __set__ output that directly feeds to output. This mostly is the sign bit of the computation. <br>
This results in that the set output is 1 if the subtraction results in a < b, meaning that rs < rt. and it is zero if rs >= rt. So it fits the logic of slt. <br>
Less input of ALU0 is connected to Set of ALU31. 

#### Final ALU
![final](final-slu.png)

## Lecture Note

Logic of SLT only cares about the least significant bit of the result
```
if R[rs] < R[rt] 
    R[rd] = 000...01 
else 
    R[rd] = 000...00 
```
* Logic of implementation: if rs < rt, then rs - rt < 0, and the sign-bit (most significant bit) is 1, else it is 0. This agrees with the conditional logic of SLT. The only problem is that we need the value to be on the LSB, but subtraction sets sign bit to MSB.<br> 
* Less input is hardwared to be zero for all the most significant bits. (when opcode 3 is chosen)<br>
* The implementation of the most significant bit is different from all other ALUs. The Set output of the most significant bit is the most significant bit of the subtraction, it is then fed into the __less__ of the least significant bit. This effectively makes the LSB to be equal to the sign bit, thus achieving our goal. <br>
* Since the subtraction is done before the Mux, we can always have the sign bit ready for all operations (this thus include our opcode 3, which is SLT). After the update from the LSB, the chain directly goes to the MUX.

#### Ripple Carry
Carry out of one is fed into the carry in of the next. 
For an 1-bit adder: 
```
Co = (A & B) | (A & Ci) | (B & Ci) 
S  = (A XOR B) XOR Ci
```
One layer of gate results in delay T. Overall delay is delay of the critical path <br>.
One adder gives maximum 2T delay, since carry out takes 2T delay, then N ripple carry adder will result in 2NT delay <br>
Need to make it better 
#### Carry Lookahead Adder 
More work is done, but in parallelism, so things can go faster. <br>
for example, when an 1-bit adder has input as two 1s, then it definitely has a carry out (no need to wait for previous adders to generate carry). Similarly, two zeros as input will definitely make carry out as zero. These cases effectively break the adder chain.<br> 
To comply to these tricks, add a __Generator Detector__ (use And gate to detect two 1s)and a __P Detector__ for each adder(uses XOR to detect two). zeros)<br> 
![look-ahead](look-ahead.png)

When A and B are not equal, in 'propagate' state, only way to have carry out is when carry in is one. <br> 
The long rectangle on the right does all the carry out calculation. ALl G,P calculations are done in parallel, and has no latency at all. 
* C4 is the carryout of the whole adder <br>. 
* C0 is the pre-determined carry in for the LSB
* C1 is the carry in for the next least significant bit adder.
* All G and P values are provided 
